# **Conversão de imagem colorida para preto e branco**

In [2]:
from PIL import Image

def converter_para_preto_e_branco(caminho_imagem, caminho_saida):
    """
    Converte uma imagem colorida para preto e branco usando a biblioteca Pillow.

    Args:
        caminho_imagem (str): O caminho para a imagem de entrada (colorida).
        caminho_saida (str): O caminho para salvar a imagem de saída (preto e branco).
    """
    try:
        # Abre a imagem
        img_colorida = Image.open(caminho_imagem)

        # Converte para preto e branco usando o modo 'L' (luminância)
        img_preto_e_branco = img_colorida.convert('L')

        # Salva a imagem convertida
        img_preto_e_branco.save(caminho_saida)
        print(f"Imagem convertida e salva em: {caminho_saida}")

    except FileNotFoundError:
        print(f"Erro: O arquivo não foi encontrado em {caminho_imagem}")
    except Exception as e:
        print(f"Ocorreu um erro: {e}")

# Exemplo de uso:
# Supondo que você tenha um arquivo 'imagem_colorida.jpg' na mesma pasta.
# O novo arquivo será salvo como 'imagem_pb.jpg'.
converter_para_preto_e_branco('free_willy_teste_img.jpg', 'imagem_pb.jpg')

Imagem convertida e salva em: imagem_pb.jpg


In [4]:
from PIL import Image, ImageFilter

def suavizar_imagem_pillow(caminho_imagem, caminho_saida, raio=2):
    """
    Suaviza uma imagem usando o filtro GaussianBlur da biblioteca Pillow.

    Args:
        caminho_imagem (str): O caminho para a imagem de entrada.
        caminho_saida (str): O caminho para salvar a imagem suavizada.
        raio (int/float): O raio do desfoque gaussiano. Um valor maior resulta em mais desfoque.
    """
    try:
        # Abre a imagem
        img = Image.open(caminho_imagem)

        # Aplica o filtro de desfoque Gaussiano
        # ImageFilter.BLUR também pode ser usado para um desfoque simples
        img_suavizada = img.filter(ImageFilter.GaussianBlur(radius=raio))

        # Salva a imagem suavizada
        img_suavizada.save(caminho_saida)
        print(f"Imagem suavizada e salva em: {caminho_saida}")

    except FileNotFoundError:
        print(f"Erro: O arquivo não foi encontrado em {caminho_imagem}")
    except Exception as e:
        print(f"Ocorreu um erro: {e}")

# Exemplo de uso:
# supondo 'minha_foto.jpg'
suavizar_imagem_pillow('free_willy_teste_img.jpg', 'minha_foto_suavizada_pillow.jpg', raio=2)

Imagem suavizada e salva em: minha_foto_suavizada_pillow.jpg


In [6]:
from PIL import Image, ImageFilter
import os

def suavizar_com_pillow(caminho_imagem, caminho_saida, tipo_filtro='GAUSSIAN_BLUR', raio=2):
    """
    Suaviza uma imagem usando diferentes filtros da biblioteca Pillow.

    Args:
        caminho_imagem (str): O caminho para a imagem de entrada.
        caminho_saida (str): O caminho para salvar a imagem suavizada.
        tipo_filtro (str): O tipo de filtro a ser usado ('BLUR', 'SMOOTH', 'SMOOTH_MORE', 'GAUSSIAN_BLUR').
        raio (int/float): O raio para o filtro Gaussiano (somente para 'GAUSSIAN_BLUR').
    """
    try:
        img = Image.open(caminho_imagem)

        if tipo_filtro == 'BLUR':
            img_suavizada = img.filter(ImageFilter.BLUR)
        elif tipo_filtro == 'SMOOTH':
            img_suavizada = img.filter(ImageFilter.SMOOTH)
        elif tipo_filtro == 'SMOOTH_MORE':
            img_suavizada = img.filter(ImageFilter.SMOOTH_MORE)
        elif tipo_filtro == 'GAUSSIAN_BLUR':
            # GAUSSIAN_BLUR aceita um 'radius'
            img_suavizada = img.filter(ImageFilter.GaussianBlur(radius=raio))
        else:
            print(f"Tipo de filtro '{tipo_filtro}' não reconhecido. Usando GAUSSIAN_BLUR padrão.")
            img_suavizada = img.filter(ImageFilter.GaussianBlur(radius=raio))

        img_suavizada.save(caminho_saida)
        print(f"Imagem suavizada ({tipo_filtro}) salva em: {caminho_saida}")

    except FileNotFoundError:
        print(f"Erro: O arquivo não foi encontrado em {caminho_imagem}")
    except Exception as e:
        print(f"Ocorreu um erro: {e}")

print("Gerando imagem de exemplo para suavização...")
suavizar_com_pillow('free_willy_teste_img.jpg', 'minha_foto_suavizada_pillow.jpg', 'SMOOTH')

Gerando imagem de exemplo para suavização...
Imagem suavizada (SMOOTH) salva em: minha_foto_suavizada_pillow.jpg


In [1]:
import sys
import os
import cv2
from PyQt5.QtWidgets import (
    QApplication, QWidget, QVBoxLayout, QComboBox, QLabel, QMessageBox
)
from PyQt5.QtGui import QIcon, QPixmap, QImage
from PyQt5.QtCore import Qt


class ImageManager:
    """Gerencia imagens para o dropdown (carregamento, cache e conversão)."""

    def __init__(self, image_paths):
        self.image_paths = image_paths
        self.cache = {}

    def get_image(self, index, width=200, height=200, process=False):
        """Retorna QPixmap de uma imagem redimensionada (e processada opcionalmente)."""
        if index < 0 or index >= len(self.image_paths):
            return None

        path = self.image_paths[index]
        if not os.path.exists(path):
            return None

        # Cache para não recarregar várias vezes
        if path not in self.cache:
            if process:
                # Exemplo: carregar com OpenCV e converter para cinza (PDI)
                img = cv2.imread(path)
                img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                img = cv2.resize(img, (width, height))

                # Converter para QImage -> QPixmap
                qimg = QImage(img.data, img.shape[1], img.shape[0], img.strides[0], QImage.Format_Grayscale8)
                pixmap = QPixmap.fromImage(qimg)
            else:
                pixmap = QPixmap(path).scaled(width, height, Qt.KeepAspectRatio, Qt.SmoothTransformation)
            self.cache[path] = pixmap

        return self.cache[path]


class ImageDropdown(QComboBox):
    """Dropdown especializado para exibir ícones de imagens."""

    def __init__(self, image_manager):
        super().__init__()
        self.image_manager = image_manager
        self._populate()

    def _populate(self):
        """Adiciona imagens no dropdown."""
        for i, path in enumerate(self.image_manager.image_paths):
            if os.path.exists(path):
                icon = QIcon(path)
                self.addItem(icon, f"Imagem {i+1}")
            else:
                self.addItem(QIcon(), f"Imagem {i+1} (não encontrada)")

    def get_selected_index(self):
        return self.currentIndex()


class MainWindow(QWidget):
    """Janela principal com preview da imagem."""

    def __init__(self, image_paths):
        super().__init__()
        self.setWindowTitle("Dropdown de Imagens - Over Engineering Edition")
        self.setGeometry(200, 200, 600, 400)

        # Instâncias principais
        self.image_manager = ImageManager(image_paths)
        self.dropdown = ImageDropdown(self.image_manager)
        self.label = QLabel("Selecione uma imagem")
        self.label.setAlignment(Qt.AlignCenter)

        # Layout
        layout = QVBoxLayout()
        layout.addWidget(self.dropdown)
        layout.addWidget(self.label)
        self.setLayout(layout)

        # Eventos
        self.dropdown.currentIndexChanged.connect(self.update_preview)

        # Preview inicial
        self.update_preview(0)

    def update_preview(self, index):
        """Atualiza o preview da imagem escolhida."""
        pixmap = self.image_manager.get_image(index, process=True)  # process=True => aplica PDI
        if pixmap:
            self.label.setPixmap(pixmap)
        else:
            QMessageBox.warning(self, "Erro", "Imagem não encontrada ou inválida.")
            self.label.setText("Erro ao carregar imagem")


if __name__ == "__main__":
    # Lista de imagens (substituir pelos arquivos existentes na pasta)
    imagens = ["free_willy_teste_img.jpg"]

    app = QApplication(sys.argv)
    janela = MainWindow(imagens)
    janela.show()
    sys.exit(app.exec_())


SystemExit: 0

C:\Users\admin\AppData\Roaming\Python\Python312\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
